In [0]:
import pyspark.sql.functions as F

In [0]:

# bronze layer 
df = spark.read.csv(
        "/Volumes/diabetes/v01/cdc-diabetes/diabetes_binary_5050split_BRFSS2015.csv",
        inferSchema = True,
        header = True
        )
df.write.mode("overwrite").format("delta").saveAsTable("health_dlt.bronze.brfs")

In [0]:
# silver layer 
df = spark.read.table("health_dlt.bronze.brfs")
df = df.select(["HighBP","HighChol","CholCheck","BMI"])
df.write.mode("overwrite").format("delta").saveAsTable("health_dlt.silver.brfs")

In [0]:
#gold layer 
df = spark.read.table("health_dlt.silver.brfs")
df = df.select(["HighBP","HighChol","CholCheck","BMI"]) \
       .withColumn(
           "BMI", 
           F.when(df.BMI < 18.5, "underWeight") \
            .when((df.BMI > 18.5) & (df.BMI < 24.9), "normal") \
            .when((df.BMI > 25) & (df.BMI < 29.9), "overWeight") \
            .otherwise("obesity") \
           ) \
       .groupBy("BMI") \
       .agg(
           F.mean(F.col("HighBP")).alias("avg_bp"),
           F.mean(F.col("HighChol")).alias("avg_chol"),
           F.mean(F.col("CholCheck")).alias("avg_chol_check")
       )
spark.sql("DROP TABLE IF EXISTS health_dlt.gold.brfs")
df.write.mode("overwrite").format("delta").saveAsTable("health_dlt.gold.brfs")

In [0]:
df = spark.read.table("health_dlt.gold.brfs")
df.display(100)